In [1]:
import onnx
import numpy as np
import onnxruntime as ort
from onnxruntime.quantization import quantize_dynamic, QuantType
from pathlib import Path
import torch
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import shutil
import logging
import os

# 禁用日志
os.environ['ORT_LOG_LEVEL'] = '3'
logging.getLogger("onnxruntime").setLevel(logging.ERROR)

MODEL_DIR = Path("../../model")
FP32_PATH = MODEL_DIR / "clip-image-encoder.onnx"
COCO_VAL_DIR = Path("../../data/dataset/coco/val2017")

# ==================== GPU 配置 ====================
print("🔧 配置 GPU 推理...")

providers = [
    ('CUDAExecutionProvider', {'device_id': 0}),
    'CPUExecutionProvider'
]

# ==================== 加载图像数据 ====================
print("\n📦 加载 COCO 图像数据...")

preprocess = transforms.Compose([
    transforms.Resize(224, interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.48145466, 0.4578275, 0.40821073),
                        (0.26862954, 0.26130258, 0.27577711)),
])

image_paths = sorted(list(COCO_VAL_DIR.glob("*.jpg")))[:500]
print(f"✅ 加载 {len(image_paths)} 张图像")

image_tensors = []
for img_p in tqdm(image_paths, desc="Loading images"):
    img = Image.open(img_p).convert("RGB")
    tensor = preprocess(img).unsqueeze(0).numpy()
    image_tensors.append(tensor)

print(f"✅ 图像加载完成")

# ==================== 节点分析 ====================
print("\n🔍 分析 ONNX 节点结构...")

model = onnx.load(str(FP32_PATH))

node_types = {}
for node in model.graph.node:
    node_types.setdefault(node.op_type, []).append(node.name)

print(f"\n📊 节点类型统计:")
for op_type, nodes in sorted(node_types.items(), key=lambda x: -len(x[1])):
    print(f"   {op_type:15s}: {len(nodes):3d} 个")

conv_nodes = node_types.get('Conv', [])
matmul_nodes = node_types.get('MatMul', [])
gemm_nodes = node_types.get('Gemm', [])
softmax_nodes = node_types.get('Softmax', [])

print(f"\n🎯 关键节点:")
print(f"   Conv: {len(conv_nodes)}, MatMul: {len(matmul_nodes)}, Gemm: {len(gemm_nodes)}, Softmax: {len(softmax_nodes)}")

# ==================== 配置 ONNX Runtime ====================
opts = ort.SessionOptions()
opts.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
opts.intra_op_num_threads = 8

# ==================== 获取 FP32 基准 ====================
print(f"\n📊 获取 FP32 基准特征（GPU）...")

sess_fp32 = ort.InferenceSession(str(FP32_PATH), opts, providers=providers)
input_name = sess_fp32.get_inputs()[0].name

print(f"   提供者: {sess_fp32.get_providers()}")

for _ in range(3):
    _ = sess_fp32.run(None, {input_name: image_tensors[0]})

print("   FP32 推理...")
fp32_feats = []
for img_tensor in tqdm(image_tensors, desc="FP32 inference"):
    output = sess_fp32.run(None, {input_name: img_tensor})[0]
    fp32_feats.append(output)

fp32_feats = np.concatenate(fp32_feats)
fp32_norm = fp32_feats / (np.linalg.norm(fp32_feats, axis=-1, keepdims=True) + 1e-7)

print(f"✅ FP32 特征: {fp32_feats.shape}")

# ==================== 定义测试函数 ====================

def get_cos_sim(model_path):
    """计算与 FP32 的余弦相似度（GPU）"""
    if not model_path.exists():
        return None

    try:
        sess = ort.InferenceSession(str(model_path), opts, providers=providers)
        input_name = sess.get_inputs()[0].name

        for _ in range(2):
            _ = sess.run(None, {input_name: image_tensors[0]})

        int8_feats = []
        for img_tensor in image_tensors:
            output = sess.run(None, {input_name: img_tensor})[0]
            int8_feats.append(output)

        int8_feats = np.concatenate(int8_feats)
        int8_norm = int8_feats / (np.linalg.norm(int8_feats, axis=-1, keepdims=True) + 1e-7)

        return np.mean(np.sum(fp32_norm * int8_norm, axis=-1))
    except:
        return None

# ==================== 策略 A: 逐个排除 Conv ====================
print("\n\n🧪 策略 A: 逐个排除 Conv 节点")
print("="*70)

temp_dir = MODEL_DIR / "temp_tests"
temp_dir.mkdir(exist_ok=True)

# 先测试全量化
test_path = temp_dir / "all_quantized.onnx"
quantize_dynamic(
    str(FP32_PATH),
    str(test_path),
    weight_type=QuantType.QInt8,
    nodes_to_exclude=[]
)

sim_baseline = get_cos_sim(test_path)

if sim_baseline is None:
    print("❌ 全量化失败，排除所有 Conv 重试")
    test_path.unlink()
    test_path = temp_dir / "all_quantized_no_conv.onnx"
    quantize_dynamic(
        str(FP32_PATH),
        str(test_path),
        weight_type=QuantType.QInt8,
        nodes_to_exclude=conv_nodes
    )
    sim_baseline = get_cos_sim(test_path)
    base_exclude = conv_nodes.copy()
    print(f"🔴 排除所有 Conv: {sim_baseline:.4f}\n")
else:
    base_exclude = []
    print(f"🔴 全量化基准: {sim_baseline:.4f}\n")

test_path.unlink()

results_conv = []

print("逐个排除 Conv 节点:\n")
for i, conv_name in enumerate(conv_nodes):
    # ✅ 累积排除：排除前 i+1 个 Conv
    exclude_list = base_exclude + conv_nodes[:i+1]
    test_path = temp_dir / f"exclude_conv_{i}.onnx"

    quantize_dynamic(
        str(FP32_PATH),
        str(test_path),
        weight_type=QuantType.QInt8,
        nodes_to_exclude=exclude_list
    )

    sim = get_cos_sim(test_path)
    results_conv.append((f"排除前 {i+1} 个 Conv", sim))

    if sim is not None:
        delta = sim - sim_baseline
        marker = "📈" if delta > 0.01 else "✓" if delta > -0.005 else "📉"
        print(f"   排除前 {i+1:2d} 个 Conv (含 {conv_name}): {sim:.4f} ({delta:+.4f}) {marker}")

    test_path.unlink()

# ==================== 策略 B: 逐个排除 MatMul ====================
print("\n\n🧪 策略 B: 逐个排除 MatMul 节点")
print("="*70)

results_matmul = []

print("\n逐个排除 MatMul 节点:\n")
for i, matmul_name in enumerate(matmul_nodes):
    # ✅ 累积排除：排除前 i+1 个 MatMul
    exclude_list = base_exclude + matmul_nodes[:i+1]
    test_path = temp_dir / f"exclude_matmul_{i}.onnx"

    quantize_dynamic(
        str(FP32_PATH),
        str(test_path),
        weight_type=QuantType.QInt8,
        nodes_to_exclude=exclude_list
    )

    sim = get_cos_sim(test_path)
    results_matmul.append((f"排除前 {i+1} 个 MatMul", sim))

    if sim is not None:
        delta = sim - sim_baseline
        marker = "📈" if delta > 0.01 else "✓" if delta > -0.005 else "📉"
        print(f"   排除前 {i+1:2d} 个 MatMul (含 {matmul_name}): {sim:.4f} ({delta:+.4f}) {marker}")

    test_path.unlink()

# ==================== 策略 C: 逐个排除 Gemm ====================
print("\n\n🧪 策略 C: 逐个排除 Gemm 节点")
print("="*70)

results_gemm = []

print("\n逐个排除 Gemm 节点:\n")
for i, gemm_name in enumerate(gemm_nodes):
    # ✅ 累积排除：排除前 i+1 个 Gemm
    exclude_list = base_exclude + gemm_nodes[:i+1]
    test_path = temp_dir / f"exclude_gemm_{i}.onnx"

    quantize_dynamic(
        str(FP32_PATH),
        str(test_path),
        weight_type=QuantType.QInt8,
        nodes_to_exclude=exclude_list
    )

    sim = get_cos_sim(test_path)
    results_gemm.append((f"排除前 {i+1} 个 Gemm", sim))

    if sim is not None:
        delta = sim - sim_baseline
        marker = "📈" if delta > 0.01 else "✓" if delta > -0.005 else "📉"
        print(f"   排除前 {i+1:2d} 个 Gemm (含 {gemm_name}): {sim:.4f} ({delta:+.4f}) {marker}")

    test_path.unlink()

# ==================== 策略 D: 逐个排除 Softmax ====================
print("\n\n🧪 策略 D: 逐个排除 Softmax 节点")
print("="*70)

results_softmax = []

print("\n逐个排除 Softmax 节点:\n")
for i, softmax_name in enumerate(softmax_nodes):
    # ✅ 累积排除：排除前 i+1 个 Softmax
    exclude_list = base_exclude + softmax_nodes[:i+1]
    test_path = temp_dir / f"exclude_softmax_{i}.onnx"

    quantize_dynamic(
        str(FP32_PATH),
        str(test_path),
        weight_type=QuantType.QInt8,
        nodes_to_exclude=exclude_list
    )

    sim = get_cos_sim(test_path)
    results_softmax.append((f"排除前 {i+1} 个 Softmax", sim))

    if sim is not None:
        delta = sim - sim_baseline
        marker = "📈" if delta > 0.01 else "✓" if delta > -0.005 else "📉"
        print(f"   排除前 {i+1:2d} 个 Softmax (含 {softmax_name}): {sim:.4f} ({delta:+.4f}) {marker}")

    test_path.unlink()

# ==================== 总结 ====================
print("\n\n" + "="*70)
print("📊 完整诊断总结")
print("="*70)

print(f"\n基准相似度: {sim_baseline:.4f}")

print(f"\n🥇 各策略最优结果:")

if results_conv:
    best_conv = max(results_conv, key=lambda x: x[1])
    print(f"\n  Conv 策略: {best_conv[0]}")
    print(f"  相似度: {best_conv[1]:.4f}")

if results_matmul:
    best_matmul = max(results_matmul, key=lambda x: x[1])
    print(f"\n  MatMul 策略: {best_matmul[0]}")
    print(f"  相似度: {best_matmul[1]:.4f}")

if results_gemm:
    best_gemm = max(results_gemm, key=lambda x: x[1])
    print(f"\n  Gemm 策略: {best_gemm[0]}")
    print(f"  相似度: {best_gemm[1]:.4f}")

if results_softmax:
    best_softmax = max(results_softmax, key=lambda x: x[1])
    print(f"\n  Softmax 策略: {best_softmax[0]}")
    print(f"  相似度: {best_softmax[1]:.4f}")

print("\n" + "="*70)

# 清理
shutil.rmtree(temp_dir)

print("\n✅ 诊断完成！")

🔧 配置 GPU 推理...

📦 加载 COCO 图像数据...
✅ 加载 500 张图像


Loading images: 100%|██████████| 500/500 [00:01<00:00, 253.67it/s]


✅ 图像加载完成

🔍 分析 ONNX 节点结构...

📊 节点类型统计:
   Reshape        : 109 个
   Transpose      :  75 个
   Add            :  61 个
   MatMul         :  61 个
   Mul            :  48 个
   Gather         :  37 个
   LayerNormalization:  26 个
   Unsqueeze      :  12 个
   Squeeze        :  12 个
   Softmax        :  12 个
   Gemm           :  12 个
   Sigmoid        :  12 个
   Conv           :   1 个
   Concat         :   1 个

🎯 关键节点:
   Conv: 1, MatMul: 61, Gemm: 12, Softmax: 12

📊 获取 FP32 基准特征（GPU）...
   提供者: ['CUDAExecutionProvider', 'CPUExecutionProvider']
   FP32 推理...


FP32 inference: 100%|██████████| 500/500 [00:00<00:00, 541.72it/s]


✅ FP32 特征: (500, 512)


🧪 策略 A: 逐个排除 Conv 节点


...

2026-02-18 11:00:22.735202604 [W:onnxruntime:, transformer_memcpy.cc:111 ApplyImpl] 144 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.


   排除前  3 个 MatMul (含 node_scaled_dot_product_attention): 0.9898 (+0.0001) ✓


2026-02-18 11:00:29.994603750 [W:onnxruntime:, transformer_memcpy.cc:111 ApplyImpl] 141 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.


   排除前  4 个 MatMul (含 node_MatMul_102): 0.9904 (+0.0008) ✓


FileNotFoundError: [Errno 2] No such file or directory: '../../model/temp_tests/exclude_matmul_3.onnx'